# Física estadística en el ensamble canónico 

Ya que contamos con un modelo microscópico, el modelo de Ising, que especifica cuáles son las *configuraciones microscópicas*, o **microestados**, del sistema, y su energía, ¡podemos hacer **física estadística**!

Por el momento, y en la mayor parte del curso, ocuparemos el **ensamble canónico**. Recordemos que esto modela un sistema en contacto con un baño térmico (o reservorio) infinito, a una temperatura fija $T$. 

[Estos conceptos no son nada sencillos, pero supondremos que ya se han tratado en el curso de Física Estadística.]

$\newcommand{\ss}{\pmb{\sigma}}$

Lo que debemos extraer de esto es que un sistema en equilibrio, a temperatura $T$, tiene cierta probabilidad de ocupar cada microestado $\ss$, dada por

$$p_T^{\mathrm{(eq)}}(\ss) \propto e^{-\beta E(\ss)},$$

donde hemos introducido la **temperatura inversa** $\beta := 1/(k_B T)$, con $k_B$ la constante de Boltzmann. La cantidad $\beta$ resulta ser más natural que pensar en términos de $T$; tomaremos unidades en las cuales $k_B = 1$.

Para la constante de proporcionalidad, introducimos la notación $Z(\beta)$, que depende de la temperatura $T$, o de $\beta$, de tal forma que las probabilidades están dadas por

$$p_T^{\mathrm{(eq)}}(\ss) = \frac{1}{Z(\beta)} e^{-\beta E(\ss)}.$$

Esta distribución de probabilidad se llama la **distribución canónica** a temperatura $T$, o la **distribución de Boltzmann(-Gibbs)**.


[1] Escribe una expresión para $Z(\beta)$, que se llama la **función de partición** a temperatura $T$. Recuerda cuál es la relación entre $Z$ y la energía libre macroscópica, $F$.

$$Z(\beta)=\sum_{\{n\}}e^{-\beta E_n(\sigma)}$$

Las cantidades de interés son cantidades macroscópicas, dadas por **promedios** sobre la distribución de Boltzmann, que denotaremos con $\langle \cdot \rangle$. En general, estas propiedades se pueden expresar en términos de la función de partición, $Z$, por lo cual $Z$ asume un papel principal en la física estadística de equilibrio.

[2] Escribe una expresión para el promedio $\langle E \rangle_\beta$ de la energía a temperatura $T$. Demuestra que se puede expresar en términos de una derivada de $Z(\beta)$.

$$\langle E \rangle_{\beta}=\frac{1}{Z}\sum_{E}E(\sigma)e^{-\beta E(\sigma)}$$

$$\langle E \rangle_{\beta}=-\frac{1}{Z}\frac{\partial}{\partial\beta}\sum_{E}e^{-\beta E(\sigma)}=-\frac{1}{Z}\frac{\partial Z}{\partial\beta}$$

$$\langle E \rangle_{\beta}=-\frac{\partial \ln Z}{\partial \beta}$$

## Cálculos exactos

En principio, podemos calcular $Z$, y los promedios, de forma exacta a partir de su definición, como sigue.

[3] (i) Escribe código que genera *todos los microestados $\ss$* del modelo de Ising cuadrado de tamaño $n \times m$, uno tras otro. [Pista: un arreglo 2D en Julia se puede manipular como si fuera de 1D; piensa en binario.]

In [2]:
int(split(bin(29),""))

5-element Array{Int64,1}:
 1
 1
 1
 0
 1

In [13]:
function menosunos(A :: Array{Int64,1})
    for i in 1:length(A)
        if A[i]==0 
            A[i]=-1 
        end
    end
    A
end

menosunos (generic function with 1 method)

In [15]:
function microestados(n :: Int64, m :: Int64)
    N=n*m
    if N>16 return 0 end
    N2=big(2)^N
    out=-ones(N2,N)
    for i in 1:N2-1
        bini=bin(i)
        out[i+1,N-length(bini)+1:N]=menosunos(int(split(bini,"")))
    end
    out
end

microestados (generic function with 1 method)

In [74]:
mi=microestados(2,2)
@time microestados(2,2);

elapsed time: 0.000182989 seconds (9728 bytes allocated)


(ii) Utiliza tu código para ir calculando $Z(\beta)$ y $\langle E \rangle$ para distintas temperaturas *para sistemas muy pequeños*. Utiliza condiciones periódicas de frontera cuando calculas $E(\ss)$. [Empieza con el sistema más pequeño y aumenta el tamaño de uno en uno...]

In [68]:
function energia(configuracion::Array{Float64,2},n::Int64,m::Int64)
    out=0.0
    configuracion=reshape(configuracion,n,m)
    for i in 1:n
        for j in 1:m
            out+=-configuracion[i,j]*(configuracion[mod1(i-1,n),j]+configuracion[mod1(i+1,n),j]+
            configuracion[i,mod1(j-1,m)]+configuracion[i,mod1(j+1,m)])
        end
    end
    out
end

energia (generic function with 2 methods)

In [69]:
energia(mi[1,:],4,4)

-64.0

In [77]:
function particion_T(T :: Float64, configuraciones :: Array{Float64,2})
    out=0.0
    for i in length(configuraciones[:,1])
        out+=e^(-energia(configuraciones[i,:],2,2)/T)
    end
    out
end

particion_T (generic function with 1 method)

In [78]:
particion_T(100.0,mi)

1.1735108709918103

(iii) ¿Hasta qué tamaño de sistema puedes llevar a cabo estos cálculos en un tiempo razonable?

(iv) Para el sistema más grande que puedas, dibuja estas cantidades en función de $T$. ¿Qué observas?

(v) ¿Qué pasa con $\langle M \rangle$? ¿Por qué? ¿Qué podríamos hacer al respecto?

[4] Hay dos valores especiales de la temperatura. 

(i) ¿Cuáles son? 

(ii) Para estas temperaturas especiales, calcula analíticamente $Z$ $\langle E \rangle$. ¿Qué debería pasar entre estas dos temperaturas?

## ¿Qué sigue? 

En principio, si calculamos $Z$ como función de $\beta$, podemos extraer las propiedades termodinámicas macroscópicas. Pero acabamos de ver que ¡*no* podemos calcular $Z$ para sistemas de tamaño interesante!

¿Cuál es la solución? Debemos encontrar una manera de calcular las cantidades de interés, como $\langle E \rangle$, *sin* tener que pasar por la vía de calcular $Z$.

Dado que estas cantidades tienen que ver con *promedios*, surge la idea de calcularlas al promediar sobre *datos* que generamos. ¿Cuáles datos? La idea es que generemos una secuencia *en el tiempo* de configuraciones $\ss^{(1)}$, $\ss^{(2)}$, etc., y que calculemos $\langle E \rangle$ como el promedio aritmético --en el sentido de la estadística-- de *los datos* $E(\ss^{(1)})$, $E(\ss^{(2)})$, etc. Por supuesto, esto nos dará sólo *una aproximación* al valor exacto. Si la generación de la secuencia ocupa un proceso azaraso, entonces *tendremos un método tipo Monte Carlo*.

Esto lo empezamos a hacer en el notebook anterior (sin darnos cuenta), pero sin contar aún con la temperatura. Ahora hemos introducido el concepto de temperatura, a través de la distribución de Boltzmann.

[5] Integra lo que hemos visto hasta ahora en el curso para ¡dar una propuesta para una solución al problema!